## Importing all the required library 

In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests, time, random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np

## Using Selenium workframe we can automate the linked in log in proccess 

In [6]:
browser = webdriver.Chrome('chromedriver.exe')
browser.get('https://www.linkedin.com/uas/login')
file = open('config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]


elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

elementID.submit()

## Below are the empty list which will be our final lists with scrapped data 

In [7]:
new = []
new2 = []
new3 = []

In [8]:
visitedprofile = []
profilesqueued = []

## Create a funtion to get the company profile, which will be saved in the list with variable named new.

In [9]:
def getnewprofileID(soup, profilesqueued):
    profileID = []
    mj = soup.find('section', {'class' : 'grid__col grid__col--lg-12 mb6'})
    all_links = mj.findAll('a', {'class':'app-aware-link'})
    for link in all_links:
        userID = link.text
        if((userID not in profilesqueued) and (userID not in visitedprofile)):
            profileID.append(userID)
    return profileID

In [10]:
company_name = []
profilesqueued2 = []

## Create a funtion to get the company name, which will be saved in the list with variable named new2. 

In [11]:
def getcompanyname(soup, profilesqueued2):
    profileID2 = []
    mj = soup.find('section', {'class' : 'grid__col grid__col--lg-12 mb6'})
    all_links = mj.findAll('div', {'class':'entity-result__primary-subtitle t-14 t-black'})
    for link in all_links:
        userID2 = link.text
        if((userID2 not in profilesqueued2) and (userID2 not in company_name)):
            profileID2.append(userID2)
    return profileID2

In [12]:
company_location = []
profilesqueued3 = []

## Create a funtion to get the company location, which will be saved in the list with variable named new3. 

In [13]:
def getcompanylocation(soup, profilesqueued2):
    profileID3 = []
    mj = soup.find('section', {'class' : 'grid__col grid__col--lg-12 mb6'})
    all_links = mj.findAll('div', {'class':'entity-result__secondary-subtitle t-14'})
    for link in all_links:
        userID3 = link.text
        if((userID3 not in profilesqueued3) and (userID3 not in company_location)):
            profileID3.append(userID3)
    return profileID3

## There are total 73 pages, we have to scarp all of them. 

In [14]:
nextlink = ['?start=10','?start=20','?start=30','?start=40','?start=50','?start=60','?start=70','?start=80','?start=90','?start=100','?start=110','?start=120','?start=130','?start=140','?start=150','?start=160','?start=170','?start=190','?start=200','?start=210','?start=220','?start=230','?start=240','?start=250','?start=260','?start=270','?start=280','?start=290','?start=300','?start=310','?start=320','?start=330','?start=340','?start=350','?start=360','?start=370','?start=380','?start=390','?start=400','?start=410','?start=420','?start=430','?start=440','?start=450','?start=460','?start=470','?start=480','?start=490','?start=500','?start=510','?start=520','?start=530','?start=540','?start=550','?start=560','?start=570','?start=580']

In [15]:
for i in nextlink:
    link = 'https://www.linkedin.com/my-items/saved-jobs/' + i
    new.extend(getnewprofileID(BeautifulSoup(browser.page_source), profilesqueued))
    new2.extend(getcompanyname(BeautifulSoup(browser.page_source), profilesqueued2))
    new3.extend(getcompanylocation(BeautifulSoup(browser.page_source), profilesqueued3))
    browser.get(link)

In [16]:
len(new)


944

In [17]:
len(new2)


0

In [18]:
len(new3)


0

## Saving the scrap data of company post in the list named as new 

In [19]:
data2 = {
        'company_post' : new
       
        }

In [20]:
df2 = pd.DataFrame(data2)

In [21]:
df2

,company_post
0,\n\n\n \n\n\n
1,\nBusiness Process Analyst (m/f/d)\n
2,\n\n\n \n\n\n
3,\nIT Business Analyst - Intelligent Customer E...
4,\n\n\n \n\n\n
...,...
939,\nCloud Data Engineer (m/w/d)\n
940,\n\n\n \n\n\n
941,\nData Engineer*\n
942,\n\n\n \n\n\n


In [22]:
df2 = df2[df2.company_post != '\n \n \n\n\n']
df2 = df2[df2.company_post != '\n\n \n\n\n\n\n\n\nApply\n\n\n']
df2 = df2[df2.company_post != '\n\n \n\n\n\n\n\n\nEasily Apply\n\n\n']

In [23]:
df2['company_post'] = df2['company_post'].apply(lambda x: x.replace('\n',''))

In [24]:
df2

,company_post
0,
1,Business Process Analyst (m/f/d)
2,
3,IT Business Analyst - Intelligent Customer Eng...
4,
...,...
939,Cloud Data Engineer (m/w/d)
940,
941,Data Engineer*
942,


In [25]:
df2.company_post.unique()

array([' ', 'Business Process Analyst (m/f/d)',
       'IT Business Analyst - Intelligent Customer Engagement (m/f/d)- Merck KGaA Darmstadt, Germany',
       'Software-Entwickler',
       'Data Scientist - All locations | Home office',
       'Werkstudent (m/w/d) im Bereich Learning & Development',
       'Business Analyst / Prozessberater:in (m/w/divers) – Digitalisierung in der Energiewirtschaft (Teilzeit möglich, 80-100%)',
       'Text data analyst - part-time [f/m/x]',
       'Werkstudent Software Engineer (m/w/d)',
       '(Senior) e-Commerce Analyst (w/m/d) FMCG',
       'Freelance Video Lecturer (m/f/d) - Advanced Data Analysis',
       'Computer Scientist - Data Science, Digital Oncology (m/f/d)- Deutsches Krebsforschungszentrum (DKFZ)',
       'Software Engineer Data, Analytics & AI',
       'Werkstudent Data Analyst / Data Science (w/m/d)',
       'Intern / Working Student: Data Engineer *',
       'Online Quality Map Analyst Germany',
       'Business Analyst (m/w/d) - IT /

In [26]:
df2 = df2[df2.company_post != ' Deutsche Bahn International']

In [27]:
df2 = df2[df2.company_post != '  ']

In [28]:
file_name_companypost = 'linked_in_companypost.xlsx'

In [29]:
file_name_location_name = 'linked_in_location_name.xlsx'

## saving scrap data of the company name in the list named as new2 

In [30]:
data = {
        'company_name' : new2
       
        }

In [31]:
df = pd.DataFrame(data)

## Saving scrap data of the comapny location in the list named as new3 

In [32]:
data3 = {
        'company_location' : new3
       
        }

In [33]:
df3 = pd.DataFrame(data3)

In [34]:
df.to_excel(file_name_location_name)

In [35]:
df2.to_excel(file_name_companypost)